In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
# Spark session 
spark = SparkSession.builder.appName("Assignment2").getOrCreate()

**PART A**
  * Part I

    Count the odd and even numbers using the file ‘integer.txt’ and download it from Quercus

In [0]:
data_rdd = spark.sparkContext.textFile("dbfs:/FileStore/shared_uploads/saridtarabay@gmail.com/integer.txt")

In [0]:
data_rdd.collect()

Out[4]: ['45687',
 '5463',
 '34443',
 '223',
 '456',
 '4667',
 '234',
 '9008',
 '1234',
 '2597',
 '6253',
 '10399',
 '3312',
 '2175',
 '20087',
 '6698',
 '31512',
 '618',
 '11698',
 '13969',
 '6575',
 '24846',
 '1313',
 '30977',
 '20927',
 '24771',
 '20293',
 '24210',
 '26005',
 '9680',
 '15297',
 '4099',
 '29625',
 '9374',
 '7645',
 '7660',
 '7025',
 '15755',
 '927',
 '22795',
 '7948',
 '29343',
 '23398',
 '5755',
 '27339',
 '19416',
 '3686',
 '21687',
 '16513',
 '32643',
 '10047',
 '11831',
 '14036',
 '11079',
 '23852',
 '11767',
 '18502',
 '28848',
 '21754',
 '4309',
 '4104',
 '4656',
 '18641',
 '20317',
 '29760',
 '8784',
 '20048',
 '26629',
 '16134',
 '15865',
 '26432',
 '22918',
 '20931',
 '15958',
 '17856',
 '13488',
 '771',
 '28089',
 '15865',
 '22489',
 '5740',
 '12871',
 '28999',
 '9373',
 '31617',
 '8185',
 '19684',
 '25075',
 '17319',
 '20320',
 '21116',
 '32064',
 '15681',
 '11990',
 '12927',
 '31383',
 '2469',
 '21500',
 '4933',
 '21212',
 '12974',
 '367',
 '19773',
 '755

In [0]:
numbers = data_rdd.flatMap(lambda line: line.split(" ")).map(int)

def odd_or_even(number):
    if number%2 ==0:
        return "even"
    else:
        return "odd"
    
odd_even_rdd = numbers.map(odd_or_even)
count = odd_even_rdd.countByValue()
print('number of odd numbers:' , count['odd'])
print('number of even numbers:' , count['even'])

number of odd numbers: 496
number of even numbers: 514


* PART 2

  Calculate the salary sum per department using the file ‘salary.txt’ and download it from Quercus. Show the department name and salary sum

In [0]:
salary_rdd = spark.sparkContext.textFile("dbfs:/FileStore/shared_uploads/saridtarabay@gmail.com/salary.txt")
salary_rdd.collect()

Out[6]: ['Sales 9136',
 'Research 13391',
 'Developer 22220',
 'QA 31888',
 'Marketing 22215',
 'Sales 45567',
 'Research 4023',
 'Developer 7262',
 'QA 5243',
 'Marketing 11425',
 'Sales 11956',
 'Research 24149',
 'Developer 18258',
 'QA 22962',
 'Marketing 28960',
 'Sales 8766',
 'Research 18343',
 'Developer 13686',
 'QA 27626',
 'Marketing 32430',
 'Sales 761',
 'Research 22690',
 'Developer 11703',
 'QA 591',
 'Marketing 15249',
 'Sales 15831',
 'Research 28373',
 'Developer 31034',
 'QA 27821',
 'Marketing 25192',
 'Sales 12640',
 'Research 15100',
 'Developer 2781',
 'QA 17773',
 'Marketing 9047',
 'Sales 20425',
 'Research 19555',
 'Developer 10498',
 'QA 18714',
 'Marketing 1209',
 'Sales 16369',
 'Research 3513',
 'Developer 3362',
 'QA 27723',
 'Marketing 25341',
 'Sales 28399',
 'Research 15762',
 'Developer 23792',
 'QA 13328',
 'Marketing 4390',
 'Sales 32523',
 'Research 16805',
 'Developer 18811',
 'QA 12468',
 'Marketing 7741',
 'Sales 23584',
 'Research 16119',
 'Dev

In [0]:
salary_split = salary_rdd.map(lambda x: x.split(","))
salary_arr = salary_split.map(lambda x: (x[0].split()[0] , int(x[0].split()[1])))
salary_per_dep = salary_arr.reduceByKey(lambda x,y: x+y)
#salary_per_dep.collect()
table_data = salary_per_dep.collect()
salary_per_dep_table = spark.createDataFrame(table_data , schema =['Department' , 'Total sum'])
salary_per_dep_table.show()

+----------+---------+
|Department|Total sum|
+----------+---------+
|     Sales|  3488491|
|  Research|  3328284|
| Developer|  3221394|
|        QA|  3360624|
| Marketing|  3158450|
+----------+---------+



* Part III

  Implement MapReduce using Pyspark on file ‘shakespeare.txt’ and download it from Quercus. Show how many times these particular words appear in the document: Shakespeare,why, Lord, Library, GUTENBERG, WILLIAM, COLLEGE and WORLD.

In [0]:
shakespear_rdd = spark.sparkContext.textFile("dbfs:/FileStore/shared_uploads/saridtarabay@gmail.com/shakespeare_1.txt")
shakespear_rdd.collect()

Out[8]: ['This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **',
 '**     Please follow the copyright guidelines in this file.     **',
 '',
 'Title: The Complete Works of William Shakespeare',
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: September 1, 2011 [EBook #100]',
 'Release Date: January, 1994',
 '',
 'Language: English',
 '',
 'Character set encoding: ASCII',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK COMPLETE WORKS--WILLIAM SHAKESPEARE ***',
 '',
 '',
 '',
 '',
 'Produced by World Library, Inc., from their Library of the Future',
 '',
 '',
 '',
 '',
 'This is the 100th Etext file presented by Project Gutenberg, and',
 'is presented in cooperation with World Library, Inc., from th

In [0]:
words_to_find = ['Shakespeare', 'why', 'Lord', 'Library', 'GUTENBERG', 'WILLIAM', 'COLLEGE','WORLD']

words_rdd = shakespear_rdd.flatMap(lambda x: x.split(" "))

# Map function
words_of_interest = words_rdd.filter(lambda word: word in words_to_find).map(lambda word: (word,1))

#Reduce function (reduce by key)

word_count = words_of_interest.reduceByKey(lambda x,y: x+y)

#word_count.collect()
table_data2 = word_count.collect()
results_table = spark.createDataFrame(table_data2 , schema = ['Word' , 'Frequency'])
results_table.show()

+-----------+---------+
|       Word|Frequency|
+-----------+---------+
|Shakespeare|       22|
|  GUTENBERG|       99|
|    WILLIAM|      115|
|      WORLD|       98|
|    COLLEGE|       98|
|        why|       91|
|       Lord|      341|
|    Library|        2|
+-----------+---------+



* PART IV
  
  Calculate the top 15 and bottom 15 words using the file ‘shakespeare.txt’ and download it from Quercus. Show 15 words with the most count and 15 words with the least count.

In [0]:

word_map = words_rdd.map(lambda word: (word.strip() , 1)).filter(lambda x: x[0]!='')
word_reduce = word_map.reduceByKey(lambda x,y: x+y)
#word_reduce.collect()

sorted_words = word_reduce.sortBy(lambda x: x[1] , ascending = False)
sorted_words.collect()

most_frequent = sorted_words.take(15)
least_frequent = sorted_words.takeOrdered(15)

In [0]:
most_frequent_table = spark.createDataFrame(most_frequent , schema = ['Word' , 'Count'])
most_frequent_table.show()

+----+-----+
|Word|Count|
+----+-----+
| the|11397|
| and| 8777|
|   I| 8556|
|  of| 7873|
|  to| 7421|
|   a| 5672|
|  my| 4913|
|  in| 4600|
| you| 4060|
| And| 3547|
|that| 3522|
|  is| 3481|
| his| 3226|
|with| 3175|
| not| 3129|
+----+-----+



In [0]:
least_frequent_table = spark.createDataFrame(least_frequent , schema = ['Word' , 'Count'])
least_frequent_table.show()

+---------+-----+
|     Word|Count|
+---------+-----+
|        "|  180|
| "AS-IS".|    1|
|   "Alas,|    1|
|   "Break|    1|
| "Brutus"|    1|
| "Brutus,|    2|
|"Caesar"?|    1|
| "Caesar,|    1|
|"Caesar."|    2|
|  "Darest|    1|
| "Defect"|    1|
|      "Do|    1|
|    "Give|    1|
| "Havoc!"|    1|
|    "Help|    1|
+---------+-----+



**PART B**
1.  Describe your data. Calculate the top 20 movies with the highest ratings and the top 15 users who provided the highest ratings.

In [0]:
path = "dbfs:/FileStore/shared_uploads/saridtarabay@gmail.com/movies.csv"

df_movies = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .option("path", path) \
  .load()


In [0]:
df_movies.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|     3|     0|
|      3|     1|     0|
|      5|     2|     0|
|      9|     4|     0|
|     11|     1|     0|
|     12|     2|     0|
|     15|     1|     0|
|     17|     1|     0|
|     19|     1|     0|
|     21|     1|     0|
|     23|     1|     0|
|     26|     3|     0|
|     27|     1|     0|
|     28|     1|     0|
|     29|     1|     0|
|     30|     1|     0|
|     31|     1|     0|
|     34|     1|     0|
|     37|     1|     0|
|     41|     2|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
grouped_ratings = df_movies.groupBy('movieId')
number_of_ratings = grouped_ratings.count()
number_of_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|     31|   15|
|     85|   18|
|     65|   11|
|     53|   12|
|     78|   14|
|     34|   11|
|     81|   16|
|     28|   12|
|     76|   11|
|     26|   14|
|     27|   15|
|     44|   16|
|     12|   17|
|     91|   14|
|     22|   20|
|     93|   10|
|     47|   14|
|      1|   13|
|     52|   14|
|     13|   16|
+-------+-----+
only showing top 20 rows



In [0]:
average_rating = df_movies.groupBy('movieId').agg({'rating':'avg'})
sorted_by_rating = average_rating.orderBy('avg(rating)' , ascending = False)
sorted_by_rating.show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|     32|2.9166666666666665|
|     90|            2.8125|
|     30|               2.5|
|     94| 2.473684210526316|
|     23| 2.466666666666667|
|     49|            2.4375|
|     29|               2.4|
|     18|               2.4|
|     52| 2.357142857142857|
|     53|              2.25|
|     62|              2.25|
|     92|2.2142857142857144|
|     46|               2.2|
|     68|2.1578947368421053|
|     87|2.1333333333333333|
|      2|2.1052631578947367|
|     69| 2.076923076923077|
|     27| 2.066666666666667|
|     88|2.0555555555555554|
|     22|              2.05|
+-------+------------------+
only showing top 20 rows



In [0]:
top_15_movies = sorted_by_rating.take(15)
top_15_movies_dataframe = spark.createDataFrame(top_15_movies , schema = ['Movie ID' , 'Avg rating'])
top_15_movies_dataframe.show()


+--------+------------------+
|Movie ID|        Avg rating|
+--------+------------------+
|      32|2.9166666666666665|
|      90|            2.8125|
|      30|               2.5|
|      94| 2.473684210526316|
|      23| 2.466666666666667|
|      49|            2.4375|
|      29|               2.4|
|      18|               2.4|
|      52| 2.357142857142857|
|      53|              2.25|
|      62|              2.25|
|      92|2.2142857142857144|
|      46|               2.2|
|      68|2.1578947368421053|
|      87|2.1333333333333333|
+--------+------------------+



In [0]:
avg_user_rating = df_movies.groupBy('userId').agg({'rating':'avg'})
sorted_by_user_rating = avg_user_rating.orderBy('avg(rating)', ascending = False)
sorted_by_user_rating.show()


+------+------------------+
|userId|       avg(rating)|
+------+------------------+
|    11|2.2857142857142856|
|    26| 2.204081632653061|
|    22|2.1607142857142856|
|    23|2.1346153846153846|
|     2|2.0652173913043477|
|    17|1.9565217391304348|
|     8|1.8979591836734695|
|    24|1.8846153846153846|
|    12|1.8545454545454545|
|     3|1.8333333333333333|
|    29| 1.826086956521739|
|    28|              1.82|
|     9|1.7924528301886793|
|    14|1.7894736842105263|
|    16|1.7777777777777777|
|     5|1.7755102040816326|
|    20|1.7659574468085106|
|    18|1.7307692307692308|
|    21|1.7083333333333333|
|     7|1.6296296296296295|
+------+------------------+
only showing top 20 rows



In [0]:
#avg ratign per user
top_15_users = sorted_by_user_rating.take(15)
top_15_users_df = spark.createDataFrame(top_15_users , schema = ['User ID' , 'Avg Rating'])
top_15_users_df.show()

+-------+------------------+
|User ID|        Avg Rating|
+-------+------------------+
|     11|2.2857142857142856|
|     26| 2.204081632653061|
|     22|2.1607142857142856|
|     23|2.1346153846153846|
|      2|2.0652173913043477|
|     17|1.9565217391304348|
|      8|1.8979591836734695|
|     24|1.8846153846153846|
|     12|1.8545454545454545|
|      3|1.8333333333333333|
|     29| 1.826086956521739|
|     28|              1.82|
|      9|1.7924528301886793|
|     14|1.7894736842105263|
|     16|1.7777777777777777|
+-------+------------------+



In [0]:
max_user_rating = df_movies.groupBy('userId').agg({'rating':'max'})
sorted_max_user_rating = max_user_rating.orderBy('max(rating)', ascending = False)
best_user_rating_df = spark.createDataFrame(sorted_max_user_rating.take(15) , schema = ['User ID' , 'Max rating' ])
best_user_rating_df.show()


+-------+----------+
|User ID|Max rating|
+-------+----------+
|     28|         5|
|     26|         5|
|     12|         5|
|     22|         5|
|      6|         5|
|     16|         5|
|      3|         5|
|     20|         5|
|      5|         5|
|     15|         5|
|      9|         5|
|     17|         5|
|      8|         5|
|     23|         5|
|      7|         5|
+-------+----------+



2. Split the dataset into train and test. Try 2 different combinations e.g. (60/40, 70/30, 75/25 and 80/20)

In [0]:
from pyspark.ml.recommendation import ALS

In [0]:
# Trying combination 60/40

training_ratio = 0.6
testing_ratio = 0.4

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

train_data , test_data = df_movies.randomSplit([training_ratio , testing_ratio] , seed=3)
model = als.fit(train_data)
predictions = model.transform(test_data)
predictions.show()



+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|     1|    28| 0.8435978|
|      7|     1|    28| 2.8506181|
|     12|     5|    28| 1.7029452|
|     13|     2|    28|  2.386222|
|     15|     1|    28| 0.9722455|
|     19|     3|    28| 1.2385218|
|     20|     1|    28| 0.5897097|
|     24|     3|    28| 1.2732593|
|     33|     1|    28| 1.0451051|
|     36|     1|    28| 1.6592953|
|     44|     1|    28| 0.7823236|
|     52|     1|    28| 1.6717854|
|     56|     1|    28|   1.00218|
|     57|     3|    28| 0.6085566|
|     60|     1|    28| 0.9056582|
|     62|     3|    28| 1.5831496|
|     82|     4|    28|0.54014313|
|     83|     1|    28| 1.8865129|
|     85|     1|    28| 1.8351246|
|     88|     2|    28| 1.2793326|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
# trying a 80/20 combination

training_ratio_2 = 0.8
testing_ratio_2 = 0.2

train_data_2 , test_data_2 = df_movies.randomSplit([training_ratio_2 , testing_ratio_2] , seed=3)

#fitting the model to the training data
model_2 = als.fit(train_data_2)

#prediction on test data
predictions_2 = model_2.transform(test_data_2)
predictions_2.show()


+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|     1|    28| 2.1032078|
|     15|     1|    28|  1.298068|
|     19|     3|    28| 1.4356452|
|     24|     3|    28| 0.6988741|
|     33|     1|    28| 1.9721751|
|     36|     1|    28| 1.3901715|
|     44|     1|    28| 1.7506129|
|     52|     1|    28| 0.8421626|
|     56|     1|    28|  1.240282|
|     57|     3|    28| 1.3548394|
|     60|     1|    28|0.97005117|
|     83|     1|    28| 2.3028364|
|     88|     2|    28| 1.4949601|
|      0|     1|    26| 1.3545141|
|      0|     1|    27|0.98920083|
|     14|     1|    26| 1.1356436|
|     16|     1|    26|0.78533506|
|     22|     2|    27|  1.364974|
|     22|     5|    26|  3.706434|
|     24|     2|    27| 1.2856618|
+-------+------+------+----------+
only showing top 20 rows



3. Explain MSE, RMSE and MAE. Compare and evaluate both of your models with evaluation metrics (RMSE or MAE), show your code and print your results. Describe which one works better and why?

  * MSE represents the mean sqaure error between the predicted class and the true class of the datapoint. This performance metric is sensitive to outliers, and is generally used when we want to penalize larger errors.
  * RMSE represents the square root of the MSE error and is generally used when we want to measure the average magnitude of errors in prediction. It informs us about the average distance between datapoints and the best fit function. 
  * MAE represents the mean absolute error between the predicted class and the true class, it provides equal weights to all errors and does not take into account the direction of the error. 

**Results Explanation**:

As expected, MAE results in a lower error estimation as it measures the absolute error between the predicted and the true class and does not take into account the direction of erros. On the other hand, RMSE results in a higher error value as it measures the average of erros between best fit function and actual datapoints and finally the MSE results in the highest error (square of the RMSE). 

In our case, we will use the RMSE error as it represents best the performance of our model in our case. In addition, error values are expressed in the same unit as the dependant variable.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator


In [0]:
#comparing model performance

evaluator_RMSE = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="rmse")
evaluator_MSE = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="mse")
evaluator_MAE = RegressionEvaluator(labelCol="rating", predictionCol="prediction", metricName="mae")

evaluators = [evaluator_RMSE , evaluator_MSE, evaluator_MAE]
predictions_list = [predictions , predictions_2]
ratios = [[0.6,0.4] , [0.8,0.2]]
      


In [0]:
for i,prediction in enumerate(predictions_list):
    print('Evaluation for ratio:' , ratios[i])
    print('RMSE:', evaluator_RMSE.evaluate(prediction))
    print('MSE:', evaluator_MSE.evaluate(prediction))
    print('MAE:', evaluator_MAE.evaluate(prediction),'\n')

Evaluation for ratio: [0.6, 0.4]
RMSE: 1.2036159101130528
MSE: 1.4486912590772723
MAE: 0.8314599303822768 

Evaluation for ratio: [0.8, 0.2]
RMSE: 1.0091415843614695
MSE: 1.0183667372875767
MAE: 0.7133230437573633 



4.  Now tune the parameters of your algorithm to get the best set of parameters. Explain different parameters of the algorithm which you have used for tuning your algorithm. Evaluate all your models again.

In [0]:
parameters = (ParamGridBuilder()
             .addGrid(als.regParam, [0.1, 0.5, 2.0])
             .addGrid(als.rank, [1,5,10])
             .addGrid(als.maxIter, [5, 10, 20])
             .addGrid(als.alpha , [0.5,1,2])
             .build())

#Tried running below command but it was takingg a long time, more than 2 hours. 
#cv = CrossValidator(estimator = als , estimatorParamMaps=parameters , evaluator=evaluator_RMSE, numFolds=5)

#use TrainValidationTest
tvs = TrainValidationSplit(estimator = als , estimatorParamMaps = parameters,evaluator = evaluator_RMSE, trainRatio = 0.8)

train , test = df_movies.randomSplit([0.8,0.2] , seed=3)



# fitting the model to the training data on the [0.8, 0.2] split as it showed a lower MSE error above
cv_model = tvs.fit(train)

In [0]:
best_model = cv_model.bestModel

best_rank = best_model._java_obj.parent().getRank()
best_alpha = best_model._java_obj.parent().getAlpha()
best_reg_param = best_model._java_obj.parent().getRegParam()
best_max_iter = best_model._java_obj.parent().getMaxIter()
best_num_block = best_model._java_obj.parent().getNumUserBlocks()

print('The best model parameters are:\nRank:',best_rank,'\nAlpha:', best_alpha,'\nRegParam:', best_reg_param,'\nmaxIter:', best_max_iter,'\nnumBlock:',best_num_block)


The best model parameters are:
Rank: 10 
Alpha: 1.0 
RegParam: 0.1 
maxIter: 20 
numBlock: 10


In [0]:
#Testing model performance on test set

prediction = best_model.transform(test_data_2)
rmse_error = evaluator_RMSE.evaluate(prediction)

print('RMSE error on test set for the best model is:' , rmse_error)

RMSE error on test set for the best model is: 1.0151288750275995


5. Calculate the top 15 movie recommendations for user id 10 and user id 14

In [0]:
def top_movie_recommendations(user_id):
    unrated_movie_by_user = df_movies.filter(df_movies.userId != user_id).groupBy('movieId').sum('rating')
    rated_by_user = df_movies.filter(df_movies.userId == user_id).select('movieId' , 'userId')
    temp_df = unrated_movie_by_user.withColumn('userId' , lit(user_id)).select('movieId' , 'userId')
    
    pred_recommendation = best_model.transform(temp_df.subtract(rated_by_user)).orderBy('prediction', ascending = False) 
    
    display(pred_recommendation.select('userId' , 'movieId' , 'prediction').take(15))
    

In [0]:
#top movie recommendation for user 10
top_movie_recommendations(10)

userId,movieId,prediction
10,92,3.321242332458496
10,9,2.8275368213653564
10,12,2.704374074935913
10,81,2.6712749004364014
10,91,2.334812641143799
10,82,2.264120101928711
10,62,2.2565178871154785
10,26,1.9144647121429443
10,95,1.744687795639038
10,23,1.6160929203033447


In [0]:
#top movie recommendation for user 14
top_movie_recommendations(14)

userId,movieId,prediction
14,85,3.0091500282287598
14,43,2.8918542861938477
14,70,2.3860840797424316
14,2,2.2388367652893066
14,77,2.152217149734497
14,58,2.086174488067627
14,90,2.0783379077911377
14,59,1.862852931022644
14,8,1.8329797983169556
14,61,1.7216458320617676
